In [ ]:
import pandas as pd
import numpy as np
import os
import random
import warnings
warnings.filterwarnings("ignore")

def run_permutation_analysis(seed_val, classes, n_permutations=1000):
    """Run only permutation testing to generate null distributions."""
    
    # Create a directory for the classification task results
    class_str = "_vs_".join(classes).replace("+", "plus").replace("-", "minus")
    output_dir = f"results_{class_str}"
    os.makedirs(output_dir, exist_ok=True)
    
    from utils import load_multimodal_data
    
    multimodal_data = load_multimodal_data(classes=classes, verbose=True)
    
    # Extract data for each modality
    eeg_data = multimodal_data['eeg']
    fmri_data = multimodal_data['fmri']
    smri_data = multimodal_data['smri']
    psychometric_data = multimodal_data['psychometric']
    
    # Sort all sub_ids in ascending order for all modalities
    eeg_sort_indices = eeg_data['sub_ids'].argsort()
    fmri_sort_indices = fmri_data['sub_ids'].argsort()
    smri_sort_indices = smri_data['sub_ids'].argsort()
    psychometric_sort_indices = psychometric_data['sub_ids'].argsort()
    
    eeg_data['sub_ids'] = eeg_data['sub_ids'][eeg_sort_indices]
    eeg_data['X'] = eeg_data['X'][eeg_sort_indices]
    eeg_data['y'] = eeg_data['y'][eeg_sort_indices]
    fmri_data['sub_ids'] = fmri_data['sub_ids'][fmri_sort_indices]
    fmri_data['X'] = fmri_data['X'][fmri_sort_indices]
    fmri_data['y'] = fmri_data['y'][fmri_sort_indices]
    smri_data['sub_ids'] = smri_data['sub_ids'][smri_sort_indices]
    smri_data['X'] = smri_data['X'][smri_sort_indices]
    smri_data['y'] = smri_data['y'][smri_sort_indices]
    psychometric_data['sub_ids'] = psychometric_data['sub_ids'][psychometric_sort_indices]
    psychometric_data['X'] = psychometric_data['X'][psychometric_sort_indices]
    psychometric_data['y'] = psychometric_data['y'][psychometric_sort_indices]
    
    from sklearn.model_selection import LeaveOneOut
    from sklearn.metrics import f1_score
    import numpy as np
    
    # Since all modalities have the same subjects, we can use any modality to get subject info
    unique_subjects = smri_data['sub_ids']
    subject_labels = smri_data['y']
    
    loo = LeaveOneOut()
    dummy_X = np.zeros((len(unique_subjects), 1))
    
    fold_splits = []
    for fold, (train_subjects_idx, test_subjects_idx) in enumerate(loo.split(dummy_X, subject_labels)):
        train_subjects = unique_subjects[train_subjects_idx]
        test_subjects = unique_subjects[test_subjects_idx]
        fold_splits.append((train_subjects, test_subjects))
    
    def get_indices_for_subjects(sub_ids, target_subjects):
        return np.isin(sub_ids, target_subjects)
    
    import xgboost as xgb
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.cluster import KMeans
    
    SEED = seed_val
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)
    
    # Store permutation results across all folds for each permutation
    permutation_predictions = {
        'sMRI': [[] for _ in range(n_permutations)],
        'Psychometric': [[] for _ in range(n_permutations)],
        'fMRI': [[] for _ in range(n_permutations)],
        'EEG': [[] for _ in range(n_permutations)]
    }
    
    # Store true labels for permutation tests (same for all modalities)
    permutation_true_labels = []
    
    WIN_STEP = 5      
    TR = 0.8     
    K_STATES_LIST_fMRI = list(range(5, 11))
    K_STATES_LIST_EEG = list(range(6, 12))
    
    for fold, (train_subjects, test_subjects) in enumerate(fold_splits):
        print(f"Processing permutations for fold {fold + 1}/{len(fold_splits)}")
        
        # Apply splits to each modality
        eeg_train_idx = get_indices_for_subjects(eeg_data['sub_ids'], train_subjects)
        eeg_test_idx = get_indices_for_subjects(eeg_data['sub_ids'], test_subjects)
        
        fmri_train_idx = get_indices_for_subjects(fmri_data['sub_ids'], train_subjects)
        fmri_test_idx = get_indices_for_subjects(fmri_data['sub_ids'], test_subjects)
        
        smri_train_idx = get_indices_for_subjects(smri_data['sub_ids'], train_subjects)
        smri_test_idx = get_indices_for_subjects(smri_data['sub_ids'], test_subjects)
        
        psychometric_train_idx = get_indices_for_subjects(psychometric_data['sub_ids'], train_subjects)
        psychometric_test_idx = get_indices_for_subjects(psychometric_data['sub_ids'], test_subjects)
        
        # Extract training and testing data for each modality
        X_train_eeg = eeg_data['X'][eeg_train_idx]
        y_train_eeg = eeg_data['y'][eeg_train_idx]
        X_test_eeg = eeg_data['X'][eeg_test_idx]
        y_test_eeg = eeg_data['y'][eeg_test_idx]
        
        X_train_fmri = fmri_data['X'][fmri_train_idx]
        y_train_fmri = fmri_data['y'][fmri_train_idx]
        X_test_fmri = fmri_data['X'][fmri_test_idx]
        y_test_fmri = fmri_data['y'][fmri_test_idx]
        
        X_train_smri = smri_data['X'][smri_train_idx]
        y_train_smri = smri_data['y'][smri_train_idx]
        X_test_smri = smri_data['X'][smri_test_idx]
        y_test_smri = smri_data['y'][smri_test_idx]
        
        X_train_psychometric = psychometric_data['X'][psychometric_train_idx]
        y_train_psychometric = psychometric_data['y'][psychometric_train_idx]
        X_test_psychometric = psychometric_data['X'][psychometric_test_idx]
        y_test_psychometric = psychometric_data['y'][psychometric_test_idx]

        # Store true labels for permutation (same across all modalities)
        permutation_true_labels.extend(y_test_smri)

        # ====== sMRI PERMUTATION TESTING ======
        print(f"  Running sMRI permutations...")
        scaler = StandardScaler()
        X_train_smri_scaled = scaler.fit_transform(X_train_smri)
        X_test_smri_scaled = scaler.transform(X_test_smri)

        counts = np.bincount(y_train_smri)
        scale_pos_weight = counts[0] / counts[1] if counts.size > 1 and counts[1] > 0 else 1.0

        for perm in range(n_permutations):
            y_train_perm = np.random.permutation(y_train_smri)
            model_perm = xgb.XGBClassifier(
                n_estimators=100, max_depth=6, learning_rate=0.05, eval_metric="logloss",
                random_state=SEED + perm, scale_pos_weight=scale_pos_weight, tree_method="hist",
                device="cuda", verbosity=0
            )
            model_perm.fit(X_train_smri_scaled, y_train_perm)
            y_pred_perm_prob = model_perm.predict_proba(X_test_smri_scaled)[:, 1]
            y_pred_perm = (y_pred_perm_prob >= 0.5).astype(int)
            permutation_predictions['sMRI'][perm].extend(y_pred_perm)

        # ====== PSYCHOMETRIC PERMUTATION TESTING ======
        print(f"  Running Psychometric permutations...")
        scaler = StandardScaler()
        X_train_psychometric_scaled = scaler.fit_transform(X_train_psychometric)
        X_test_psychometric_scaled = scaler.transform(X_test_psychometric)

        counts = np.bincount(y_train_psychometric)
        scale_pos_weight = counts[0] / counts[1] if counts.size > 1 and counts[1] > 0 else 1.0
        
        for perm in range(n_permutations):
            y_train_perm = np.random.permutation(y_train_psychometric)
            model_perm = xgb.XGBClassifier(
                n_estimators=100, max_depth=6, learning_rate=0.05, eval_metric="logloss",
                random_state=SEED + perm, scale_pos_weight=scale_pos_weight, tree_method="hist",
                device="cuda", verbosity=0
            )
            model_perm.fit(X_train_psychometric_scaled, y_train_perm)
            y_pred_perm_prob = model_perm.predict_proba(X_test_psychometric_scaled)[:, 1]
            y_pred_perm = (y_pred_perm_prob >= 0.5).astype(int)
            permutation_predictions['Psychometric'][perm].extend(y_pred_perm)

        # ====== fMRI PERMUTATION TESTING ======
        print(f"  Running fMRI permutations...")
        from scipy.stats import kruskal, mannwhitneyu, shapiro, levene, f_oneway, ttest_ind
        from sklearn.pipeline import make_pipeline
        from sklearn.linear_model import LogisticRegression

        STEP_SEC = WIN_STEP * TR
        NORMALITY_ALPHA = 0.05
        VARIANCE_ALPHA = 0.05
        MIN_SAMPLE_SIZE = 3
        P_VALUE_SMOOTHING = 1e-10
        WEIGHT_POWER = 2.0

        def test_normality_and_variance(groups, state_id, fold_id):
            test_details = {
                'state': state_id, 'fold': fold_id, 'n_groups': len(groups),
                'group_sizes': [len(g) for g in groups], 'use_parametric': False,
                'reason': ''
            }
            if any(len(g) < MIN_SAMPLE_SIZE for g in groups):
                test_details['reason'] = f'Sample size too small (min {MIN_SAMPLE_SIZE} required)'
                return False, test_details
            normality_pvals = [shapiro(g)[1] if len(g) >= 3 else 0.0 for g in groups]
            if not all(p > NORMALITY_ALPHA for p in normality_pvals):
                test_details['reason'] = 'One or more groups not normally distributed'
                return False, test_details
            try:
                if levene(*groups)[1] <= VARIANCE_ALPHA:
                    test_details['reason'] = 'Unequal variances detected'
                    return False, test_details
            except:
                test_details['reason'] = 'Could not test variance homogeneity'
                return False, test_details
            test_details['reason'] = 'All parametric assumptions satisfied'
            test_details['use_parametric'] = True
            return True, test_details

        def choose_statistical_test_adaptive(groups, state_id, fold_id):
            if len(groups) < 2 or any(g.size == 0 for g in groups) or np.std(np.concatenate(groups)) <= 1e-10:
                return 'None', 1.0, 0.0
            use_parametric, _ = test_normality_and_variance(groups, state_id, fold_id)
            try:
                if use_parametric:
                    test_func = ttest_ind if len(groups) == 2 else f_oneway
                    stat, p_val = test_func(*groups)
                    return ('t-test' if len(groups) == 2 else 'ANOVA'), p_val, stat
                else:
                    test_func = mannwhitneyu if len(groups) == 2 else kruskal
                    stat, p_val = test_func(*groups)
                    return ('Mann-Whitney' if len(groups) == 2 else 'Kruskal-Wallis'), p_val, stat
            except Exception:
                return 'None', 1.0, 0.0

        def compute_pvalue_weights(p_values, smoothing=P_VALUE_SMOOTHING, power=WEIGHT_POWER):
            smoothed_p_values = np.array(p_values) + smoothing
            inverse_p = (1.0 / smoothed_p_values) ** power
            return inverse_p / np.sum(inverse_p)

        def weighted_prediction(probabilities, weights):
            weighted_prob = np.average(probabilities, axis=1, weights=weights)
            return (weighted_prob > 0.5).astype(int), weighted_prob

        def compute_subject_dwell_times(X_data, y_data, sub_ids_data, target_subjects, kmeans_model, k_states):
            subject_features, subject_labels, processed_subjects = [], [], []
            for subject in target_subjects:
                subject_mask = sub_ids_data == subject
                if not np.any(subject_mask): continue
                subject_fmri = X_data[subject_mask]
                subject_preds = kmeans_model.predict(subject_fmri)
                subject_dwell_times = np.bincount(subject_preds, minlength=k_states) * STEP_SEC
                subject_features.append(subject_dwell_times)
                subject_labels.append(y_data[subject_mask][0])
                processed_subjects.append(subject)
            return np.array(subject_features), np.array(subject_labels), processed_subjects

        # Pre-compute K-means models and subject features for efficiency
        kmeans_models = {}
        X_train_fmri_subjects_all = {}
        X_test_fmri_subjects_all = {}
        
        for K_STATES in K_STATES_LIST_fMRI:
            kmeans = KMeans(n_clusters=K_STATES, random_state=SEED, n_init="auto").fit(X_train_fmri)
            kmeans_models[K_STATES] = kmeans
            
            X_train_fmri_subjects, y_train_fmri_subjects, _ = compute_subject_dwell_times(
                X_train_fmri, y_train_fmri, fmri_data['sub_ids'][fmri_train_idx], train_subjects, kmeans, K_STATES)
            X_test_fmri_subjects, y_test_fmri_subjects, _ = compute_subject_dwell_times(
                X_test_fmri, y_test_fmri, fmri_data['sub_ids'][fmri_test_idx], test_subjects, kmeans, K_STATES)
            
            X_train_fmri_subjects_all[K_STATES] = X_train_fmri_subjects
            X_test_fmri_subjects_all[K_STATES] = X_test_fmri_subjects

        for perm in range(n_permutations):
            y_train_perm = np.random.permutation(y_train_fmri_subjects)
            
            perm_results_per_k = []
            perm_fold_p_values = []

            for K_STATES in K_STATES_LIST_fMRI:
                X_train_subjects = X_train_fmri_subjects_all[K_STATES]
                X_test_subjects = X_test_fmri_subjects_all[K_STATES]
                
                # Feature selection with permuted labels
                pvals = []
                for state in range(K_STATES):
                    groups = [X_train_subjects[y_train_perm == label, state] for label in np.unique(y_train_perm)]
                    _, p_value, _ = choose_statistical_test_adaptive(groups, state, fold)
                    pvals.append(p_value)
                
                best_state_idx = np.argmin(pvals)
                perm_fold_p_values.append(pvals[best_state_idx])
                
                clf_perm = make_pipeline(MinMaxScaler(), LogisticRegression(penalty=None, class_weight="balanced", random_state=SEED + perm))
                clf_perm.fit(X_train_subjects[:, [best_state_idx]], y_train_perm)
                
                perm_prob = clf_perm.predict_proba(X_test_subjects[:, [best_state_idx]])[:, 1]
                perm_results_per_k.append({'probabilities': perm_prob})

            perm_ensemble_weights = compute_pvalue_weights(perm_fold_p_values)
            perm_all_probabilities = np.column_stack([res['probabilities'] for res in perm_results_per_k])
            perm_pred, _ = weighted_prediction(perm_all_probabilities, perm_ensemble_weights)
            
            permutation_predictions['fMRI'][perm].extend(perm_pred)

        # ====== EEG PERMUTATION TESTING ======
        print(f"  Running EEG permutations...")
        EEG_WINDOW_SIZE_SEC = 4.0
        EEG_OVERLAP_PERCENT = 0.75
        STEP_SEC = EEG_WINDOW_SIZE_SEC * (1 - EEG_OVERLAP_PERCENT)
        
        def compute_eeg_subject_features(X_data, y_data, sub_ids_data, target_subjects, kmeans_model, k_states):
            subject_features, subject_labels, processed_subjects = [], [], []
            for subject in target_subjects:
                subject_mask = sub_ids_data == subject
                if not np.any(subject_mask): continue
                subject_eeg = X_data[subject_mask]
                subject_eeg_flat = subject_eeg.reshape(subject_eeg.shape[0], -1)
                subject_preds = kmeans_model.predict(subject_eeg_flat)
                subject_dwell_times = np.bincount(subject_preds, minlength=k_states) * STEP_SEC
                subject_features.append(subject_dwell_times)
                subject_labels.append(y_data[subject_mask][0])
                processed_subjects.append(subject)
            return np.array(subject_features), np.array(subject_labels), processed_subjects

        # Pre-process EEG data
        X_train_eeg = X_train_eeg[:, np.tril_indices(X_train_eeg.shape[1], k=-1)[0], np.tril_indices(X_train_eeg.shape[1], k=-1)[1], :]
        X_test_eeg = X_test_eeg[:, np.tril_indices(X_test_eeg.shape[1], k=-1)[0], np.tril_indices(X_test_eeg.shape[1], k=-1)[1], :]
        
        # Pre-compute K-means models and subject features for efficiency
        kmeans_eeg_models = {}
        X_train_eeg_subjects_all = {}
        X_test_eeg_subjects_all = {}
        
        for K_STATES in K_STATES_LIST_EEG:
            X_train_eeg_flat = X_train_eeg.reshape(X_train_eeg.shape[0], -1)
            kmeans_eeg = KMeans(n_clusters=K_STATES, random_state=SEED, n_init="auto").fit(X_train_eeg_flat)
            kmeans_eeg_models[K_STATES] = kmeans_eeg
            
            X_train_eeg_subjects, y_train_eeg_subjects, _ = compute_eeg_subject_features(
                X_train_eeg, y_train_eeg, eeg_data['sub_ids'][eeg_train_idx], train_subjects, kmeans_eeg, K_STATES)
            X_test_eeg_subjects, y_test_eeg_subjects, _ = compute_eeg_subject_features(
                X_test_eeg, y_test_eeg, eeg_data['sub_ids'][eeg_test_idx], test_subjects, kmeans_eeg, K_STATES)
            
            X_train_eeg_subjects_all[K_STATES] = X_train_eeg_subjects
            X_test_eeg_subjects_all[K_STATES] = X_test_eeg_subjects

        for perm in range(n_permutations):
            y_train_perm = np.random.permutation(y_train_eeg_subjects)
            
            perm_results_per_k = []
            perm_fold_p_values = []

            for K_STATES in K_STATES_LIST_EEG:
                X_train_subjects = X_train_eeg_subjects_all[K_STATES]
                X_test_subjects = X_test_eeg_subjects_all[K_STATES]
                
                # Feature selection with permuted labels
                pvals = []
                for state in range(K_STATES):
                    groups = [X_train_subjects[y_train_perm == label, state] for label in np.unique(y_train_perm)]
                    _, p_value, _ = choose_statistical_test_adaptive(groups, state, fold)
                    pvals.append(p_value)
                
                best_state_idx = np.argmin(pvals)
                perm_fold_p_values.append(pvals[best_state_idx])
                
                clf_perm = make_pipeline(MinMaxScaler(), LogisticRegression(penalty=None, class_weight="balanced", random_state=SEED + perm))
                clf_perm.fit(X_train_subjects[:, [best_state_idx]], y_train_perm)
                
                perm_prob = clf_perm.predict_proba(X_test_subjects[:, [best_state_idx]])[:, 1]
                perm_results_per_k.append({'probabilities': perm_prob})

            perm_ensemble_weights = compute_pvalue_weights(perm_fold_p_values)
            perm_all_probabilities = np.column_stack([res['probabilities'] for res in perm_results_per_k])
            perm_pred, _ = weighted_prediction(perm_all_probabilities, perm_ensemble_weights)
            
            permutation_predictions['EEG'][perm].extend(perm_pred)

    # Compute F1 scores for each permutation
    permutation_f1_results = {
        'sMRI': [], 'Psychometric': [], 'fMRI': [], 'EEG': []
    }
    
    print("\nComputing F1 scores for permutation distributions...")
    for mod in ['sMRI', 'Psychometric', 'fMRI', 'EEG']:
        for perm in range(n_permutations):
            f1 = f1_score(permutation_true_labels, permutation_predictions[mod][perm], average='macro', zero_division=0)
            permutation_f1_results[mod].append(f1)

    return {
        'output_dir': output_dir,
        'permutation_f1_results': permutation_f1_results,
        'permutation_true_labels': permutation_true_labels,
        'permutation_predictions': permutation_predictions
    }

def load_observed_results(results_folder):
    """Load observed F1 scores from existing CSV files."""
    observed_results = {}
    
    # Define all classification tasks
    tasks = [
        ("N", "A+P-"),
        ("N", "A+P+"), 
        ("A+P-", "A+P+"),
        ("N", "A+P-", "A+P+")
    ]
    
    for task in tasks:
        class_str = "_vs_".join(task).replace("+", "plus").replace("-", "minus")
        csv_path = os.path.join(results_folder, f"results_{class_str}", "multimodal_results_aggregated.csv")
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            
            # Extract F1 scores for individual modalities
            task_results = {}
            for modality in ['sMRI', 'Psychometric', 'fMRI', 'EEG']:
                modality_row = df[df['combination'] == modality]
                if not modality_row.empty:
                    task_results[modality] = modality_row['f1_macro_mean'].iloc[0]
                else:
                    task_results[modality] = None
            
            observed_results[class_str] = task_results
            print(f"Loaded observed F1 scores for {class_str}")
        else:
            print(f"Warning: Could not find results for {class_str}")
    
    return observed_results

def run_comprehensive_permutation_analysis():
    """Run permutation testing for all classification tasks and compare with observed results."""
    
    print("=" * 120)
    print("COMPREHENSIVE PERMUTATION TESTING ANALYSIS")
    print("=" * 120)
    
    # Load observed results from CSV files
    results_folder = "final_results_v2"  # Adjust path as needed
    observed_results = load_observed_results(results_folder)
    
    # Classification tasks to test
    tasks = [
        ("N", "A+P-"),
        ("N", "A+P+"), 
        ("A+P-", "A+P+"),
        ("N", "A+P-", "A+P+")  # Uncomment if you want to test 3-way classification
    ]
    
    all_permutation_results = {}
    
    for task in tasks:
        class_str = "_vs_".join(task).replace("+", "plus").replace("-", "minus")
        print(f"\n{'='*80}")
        print(f"RUNNING PERMUTATION TESTING FOR: {class_str}")
        print('='*80)
        
        # Run permutation analysis
        perm_results = run_permutation_analysis(seed_val=42, classes=task, n_permutations=1000)
        all_permutation_results[class_str] = perm_results
    
    # ===== STATISTICAL ANALYSIS =====
    print("\n" + "=" * 120)
    print("PERMUTATION TEST RESULTS SUMMARY")
    print("=" * 120)
    
    summary_results = []
    
    for task_name, perm_data in all_permutation_results.items():
        print(f"\n📊 TASK: {task_name.upper()}")
        print("-" * 60)
        
        for modality in ['sMRI', 'Psychometric', 'fMRI', 'EEG']:
            # Get observed F1 score
            if task_name in observed_results and modality in observed_results[task_name]:
                observed_f1 = observed_results[task_name][modality]
            else:
                observed_f1 = None
                
            if observed_f1 is None:
                print(f"  {modality}: No observed data available")
                continue
                
            # Get permutation distribution
            perm_f1_dist = np.array(perm_data['permutation_f1_results'][modality])
            
            # Calculate p-value (corrected with +1)
            num_greater_equal = np.sum(perm_f1_dist >= observed_f1)
            p_value = (num_greater_equal + 1) / (len(perm_f1_dist) + 1)
            
            # Statistical summary
            perm_mean = np.mean(perm_f1_dist)
            perm_std = np.std(perm_f1_dist)
            
            print(f"  {modality:12}: Obs={observed_f1:.4f}, Perm μ={perm_mean:.4f}±{perm_std:.4f}, p={p_value:.4f} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''}")
            
            # Store for summary table
            summary_results.append({
                'Task': task_name,
                'Modality': modality,
                'Observed_F1': observed_f1,
                'Permutation_Mean_F1': perm_mean,
                'Permutation_Std_F1': perm_std,
                'p_value': p_value,
                'Significant': p_value < 0.05,
                'Effect_Size': (observed_f1 - perm_mean) / perm_std if perm_std > 0 else 0
            })
    
    # Create comprehensive summary DataFrame
    summary_df = pd.DataFrame(summary_results)
    
    # ===== FINAL SUMMARY TABLE =====
    print("\n" + "=" * 140)
    print("COMPREHENSIVE PERMUTATION TEST RESULTS")
    print("=" * 140)
    
    # Format for display
    display_summary = summary_df.copy()
    display_summary['Observed_F1'] = display_summary['Observed_F1'].apply(lambda x: f"{x:.4f}")
    display_summary['Permutation_Mean_F1'] = display_summary['Permutation_Mean_F1'].apply(lambda x: f"{x:.4f}")
    display_summary['Permutation_Std_F1'] = display_summary['Permutation_Std_F1'].apply(lambda x: f"{x:.4f}")
    display_summary['p_value'] = display_summary['p_value'].apply(lambda x: f"{x:.4f}")
    display_summary['Effect_Size'] = display_summary['Effect_Size'].apply(lambda x: f"{x:.4f}")
    display_summary['Significance'] = display_summary['Significant'].apply(lambda x: '***' if x else 'ns')
    
    print(display_summary[['Task', 'Modality', 'Observed_F1', 'Permutation_Mean_F1', 'Permutation_Std_F1', 'p_value', 'Significance', 'Effect_Size']].to_string(index=False))
    
    # Save results
    results_output_dir = "permutation_test_analysis"
    os.makedirs(results_output_dir, exist_ok=True)
    
    summary_df.to_csv(os.path.join(results_output_dir, 'permutation_test_comprehensive_results.csv'), index=False)
    
    # Save permutation distributions
    for task_name, perm_data in all_permutation_results.items():
        perm_dist_path = os.path.join(results_output_dir, f'permutation_distributions_{task_name}.npz')
        np.savez_compressed(perm_dist_path, **perm_data['permutation_f1_results'])
    
    print(f"\n\nRESULTS SAVED TO: '{results_output_dir}'")
    print("- Comprehensive summary: permutation_test_comprehensive_results.csv")
    print("- Permutation distributions: permutation_distributions_*.npz")
    
    # ===== SIGNIFICANCE SUMMARY =====
    print("\n" + "=" * 80)
    print("SIGNIFICANCE SUMMARY")
    print("=" * 80)
    
    sig_count = summary_df[summary_df['Significant']].groupby('Modality').size()
    total_count = summary_df.groupby('Modality').size()
    
    for modality in ['sMRI', 'Psychometric', 'fMRI', 'EEG']:
        sig = sig_count.get(modality, 0)
        total = total_count.get(modality, 0)
        print(f"{modality:12}: {sig}/{total} tasks significant ({sig/total*100:.1f}%)")
    
    return summary_df, all_permutation_results

# ===== MAIN EXECUTION =====
if __name__ == "__main__":
    summary_results, permutation_data = run_comprehensive_permutation_analysis()

RUNNING MULTIMODAL ANALYSIS WITH CORRECTED PERMUTATION TESTING

RUNNING ANALYSIS 1/1 WITH SEED 9
Loading all modality data...
Shape of concatenated data for 02: (358, 32, 32, 5)
Labels shape for 02: (358,), First label: 0
Shape of concatenated data for 40: (358, 32, 32, 5)
Labels shape for 40: (358,), First label: 2
Shape of concatenated data for 75: (358, 32, 32, 5)
Labels shape for 75: (358,), First label: 1
Shape of concatenated data for 47: (358, 32, 32, 5)
Labels shape for 47: (358,), First label: 1
Shape of concatenated data for 32: (358, 32, 32, 5)
Labels shape for 32: (358,), First label: 2
Shape of concatenated data for 38: (358, 32, 32, 5)
Labels shape for 38: (358,), First label: 1
Shape of concatenated data for 25: (358, 32, 32, 5)
Labels shape for 25: (358,), First label: 0
Shape of concatenated data for 53: (358, 32, 32, 5)
Labels shape for 53: (358,), First label: 1
Shape of concatenated data for 64: (358, 32, 32, 5)
Labels shape for 64: (358,), First label: 1
Shape of c